In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def chrome_setting():
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless') # 크롬 탭이 나타나지 않게 옵션 조정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage') # 공유 메모리를 담당하는 디렉토리 사용 x
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [15]:
# 웹드라이버 오픈
driver = chrome_setting()
driver.implicitly_wait(10)

# 장르 - 발라드 - 인기순 url
# 장르, 인기순, 인덱스 넘버 확인
url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
lyrics_list = []
singer_list = []
date_list = []
like_list = []
genre_list = []

# range값 지정
for x in range(1,2):
    for y in range(1,2):
        for z in range(1, 10):
            time.sleep(3)
            click_element3 = WebDriverWait(driver, 2) \
                .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{z}]/td[4]/div/a')))
            click_element3.click()

            try:
                # 제목
                title = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))

                # 가사
                lyrics = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
                
                # 가수
                singer = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
                
                # 발매일
                date = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)")))
                
                # 좋아요 수
                like = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
                
                # 장르
                genre = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)")))
                
                title_list.append(title.text)
                lyrics_list.append(lyrics.text)
                singer_list.append(singer.text)
                date_list.append(date.text)
                like_list.append(like.text)
                genre_list.append(genre.text)

            except:
                print(x,y,z,"에서 에러 발생.")
            
            driver.back()
            
        time.sleep(1)
        click_element2 = WebDriverWait(driver, 2) \
            .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{y}]')))
        click_element2.click()

    time.sleep(1)
    
    # 이 코드는 인덱스 501이상부터 크롤링 할 때 ######
    # a_tag = 'a[2]'
    ####################################

    # 이 코드는 처음부터 크롤링 할 때 #########
    a_tag = 'a' if x==1 else 'a[2]'
    ####################################

    click_element1 = WebDriverWait(driver, 2) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pageObjNavgation"]/div/{a_tag}')))
    click_element1.click()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:01<00:00, 6.46MB/s]


In [16]:
raw_result = {'제목': title_list,
              '가사': lyrics_list,
              '가수' : singer_list,
              '발매일' : date_list,
              '좋아요수' : like_list,
              '장르': genre_list}

result = pd.DataFrame(raw_result)

In [17]:
print(result.shape)
result.tail(10)

(9, 6)


,제목,가사,가수,발매일,좋아요수,장르
0,Monologue,다 잊었다는 거짓말\n또 해 버렸죠\n내 마음에 그대란 사람\n없다고 했죠\n너무나...,테이,2022.09.18,"66,060",발라드
1,너의 모든 순간,이윽고 내가 한눈에\n너를 알아봤을 때\n모든 건 분명 달라지고 있었어\n내 세상은...,성시경,2014.02.12,"243,087","발라드, 국내드라마"
2,잘가요,미안해 마요 이제야 난 깨달아요\n내 절대 그대 짝이 아님을\n괜찮을게요 영혼 밖에...,주호,2022.12.04,"18,698",발라드
3,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면\n그 사람 손길이 자꾸 생각이 난다\n붙잡지 못하고 가...,임영웅,2021.10.11,"185,587","발라드, 국내드라마"
4,해요 (2022),그녀와 나는요 그땐 참 어렸어요\n많이 사랑했고 때론 많이 다퉜었죠\n지금 생각하면...,#안녕,2022.06.07,"63,805","발라드, 인디음악"
5,사랑인가 봐,너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...,멜로망스,2022.02.18,"169,582","발라드, 국내드라마"
6,내가 아니라도,사랑이었다 별거 없던 내 하루에\n빛이 돼준 단한 사람\n나보다 나를 더 아껴 주던...,주호,2022.03.25,"86,929",발라드
7,그대를 알고,내가 좋아하는 걸 더 좋아했었죠\n괜한 투정 부려도 화낸 적 없었죠\n나쁜 일이 있...,송하예,2022.12.13,"13,637",발라드
8,그때 그 순간 그대로 (그그그),잘 지냈지? 조금은 어색해 요즘 좋아 보여 인사 나누며\n사실 궁금한 게 너무 많았...,WSG워너비 (가야G),2022.07.09,"122,676",발라드


In [54]:
# csv 파일로 save
# result.to_csv("data/melon_ballad_16501_17000.csv", index=False)